**Clone the Repository and change the directory**

In [1]:
!git clone https://github.com/Kalash1106/ML_GC_2K24.git

Cloning into 'ML_GC_2K24'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 102 (delta 16), reused 2 (delta 2), pack-reused 65
Receiving objects: 100% (102/102), 75.18 KiB | 4.42 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [2]:
cd ML_GC_2K24

/kaggle/working/ML_GC_2K24


**Load the Config File and set the correct paths**

In [3]:
import json

# Opening JSON file
f = open('config.json')
params = json.load(f)

In [4]:
params

{'train_image_folder': 'data/KCDH2024_Training_Input_10K',
 'gt_file': 'data/KCDH2024_Training_GroundTruth.csv',
 'mapping_file': 'utility/disease_id.json',
 'eval_image_folder': 'data/KCDH2024_Test_Input',
 'eval_labels': 'data/eval_labels.csv',
 'model_path': 'weights/ResNet18',
 'submission_csv_path': 'submission.csv',
 'img_size': 224,
 'test_size': 0.15,
 'pretrained': 1,
 'num_epochs': 10,
 'train_batch_size': 32,
 'test_batch_size': 32,
 'eval_batch_size': 32}

In [12]:
params['train_image_folder'] = "/kaggle/input/aiml-general-championship/KCDH2024_Training_Input_10K/KCDH2024_Training_Input_10K"
params['gt_file'] = "/kaggle/working/ML_GC_2K24/data/KCDH2024_Training_GroundTruth.csv"
params['mapping_file'] = "/kaggle/working/ML_GC_2K24/utility/disease_id.json"
params['eval_image_folder'] = "/kaggle/input/aiml-general-championship/KCDH2024_Test_Input/KCDH2024_Test_Input"
params['eval_labels'] = "/kaggle/working/ML_GC_2K24/data/eval_labels.csv"
params['train_batch_size'] = 64

**Creating the Dataloader**

In [13]:
%%time
from utility.data_gen import DataUtility

DataClass = DataUtility(params['train_image_folder'], params['gt_file'], params['mapping_file'], params['img_size'], params['test_size'])
DataClass.make_dataloader(train_batch_size = params['train_batch_size'])

/kaggle/working/ML_GC_2K24/utility/data_gen.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  results.append({"image": row[0], "class_id": class_id})


CPU times: user 2.24 s, sys: 8.41 ms, total: 2.25 s
Wall time: 2.25 s


{'train': <torch.utils.data.dataloader.DataLoader at 0x79cb4f00f280>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x79cb4f00e3b0>}

In [14]:
#The sum should be 9600
print(len(DataClass.dataset['train']), len(DataClass.dataset['test']))

8160 1440


**Defining the Model & Criterion (loss function)**

In [8]:
import torch
from torchvision import models    
num_classes = 7
criterion = torch.nn.CrossEntropyLoss()
        
if NV_binary:
    num_classes = 1
    criterion = torch.nn.BCEWithLogitsLoss()
        
    model = models.resnet18(pretrained=False)

    #Adding the FC layer on top
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, num_classes)
    torch.nn.init.xavier_uniform_(model.fc.weight)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 148MB/s]


Parameter containing:
tensor([[ 0.0335,  0.0836,  0.0963,  ...,  0.0850,  0.0124,  0.0008],
        [ 0.0781,  0.0692,  0.0579,  ..., -0.0084,  0.0345,  0.0224],
        [ 0.0039,  0.0595,  0.0255,  ...,  0.0792,  0.0525,  0.0372],
        ...,
        [ 0.0353,  0.0569, -0.0902,  ..., -0.0731,  0.0365, -0.0307],
        [-0.0733,  0.0543,  0.0492,  ..., -0.1022,  0.0106,  0.0384],
        [ 0.0155, -0.0122,  0.0811,  ...,  0.0089,  0.0649,  0.0854]],
       requires_grad=True)

**Training the Model**

In [19]:
#Checking the GPU
!nvidia-smi

Tue Mar 12 06:17:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              31W / 250W |   3148MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [16]:
from main import train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model = train(params, DataClass, model, criterion, device)

Epoch 1/10, Loss: 0.8518
Epoch 2/10, Loss: 0.6453
Epoch 3/10, Loss: 0.5753
Epoch 4/10, Loss: 0.5324
Epoch 5/10, Loss: 0.5006
Epoch 6/10, Loss: 0.4713
Epoch 7/10, Loss: 0.4592
Epoch 8/10, Loss: 0.4380
Epoch 9/10, Loss: 0.4367
Epoch 10/10, Loss: 0.4247


**Evaluating & Saving the Model**

In [17]:
from main import test

test(DataClass, trained_model, device)

Accuracy on test set: 0.8187
Accuracy on test set: 0.8187
Precision: [0.59482759 0.87962085 0.66666667 0.62857143 0.66935484 0.625
 0.88888889]
Recall: [0.45394737 0.95473251 0.77777778 0.43137255 0.54605263 0.27777778
 0.69565217]
F1-Score: [0.51492537 0.91563888 0.71794872 0.51162791 0.60144928 0.38461538
 0.7804878 ]


In [18]:
model_path = "/kaggle/working/ResNet18"
torch.save(trained_model, model_path)

**Making the Submission CSV File**

In [20]:
from utility.eval import make_submission_csv

make_submission_csv(params,trained_model, device, csv_file_path = '/kaggle/working/submission.csv', batch_size = 128)